<a href="https://colab.research.google.com/github/SUBHADEEP1710/Movie-Genre-Classification/blob/master/Movie_Genre_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR']='/content'

In [2]:
!kaggle datasets download -d hijest/genre-classification-dataset-imdb

 67% 28.0M/41.7M [00:00<00:00, 126MB/s] 
100% 41.7M/41.7M [00:00<00:00, 101MB/s]


In [3]:
!unzip genre-classification-dataset-imdb.zip

Archive:  genre-classification-dataset-imdb.zip
  inflating: Genre Classification Dataset/description.txt  
  inflating: Genre Classification Dataset/test_data.txt  
  inflating: Genre Classification Dataset/test_data_solution.txt  
  inflating: Genre Classification Dataset/train_data.txt  


In [4]:
# Load data from the text files
with open('/content/Genre Classification Dataset/description.txt', 'r') as description_file:
    description = description_file.read()

with open('/content/Genre Classification Dataset/train_data.txt', 'r') as train_data_file:
    train_data = train_data_file.read()

with open('/content/Genre Classification Dataset/test_data.txt', 'r') as test_data_file:
    test_data = test_data_file.read()

with open('/content/Genre Classification Dataset/test_data_solution.txt', 'r') as test_data_solution_file:
    test_data_solution = test_data_solution_file.read()



In [ ]:
print(description)

Train data:
ID ::: TITLE ::: GENRE ::: DESCRIPTION
ID ::: TITLE ::: GENRE ::: DESCRIPTION
ID ::: TITLE ::: GENRE ::: DESCRIPTION
ID ::: TITLE ::: GENRE ::: DESCRIPTION

Test data:
ID ::: TITLE ::: DESCRIPTION
ID ::: TITLE ::: DESCRIPTION
ID ::: TITLE ::: DESCRIPTION
ID ::: TITLE ::: DESCRIPTION

Source:
ftp://ftp.fu-berlin.de/pub/misc/movies/database/


In [5]:



train_lines = train_data.split('\n')
test_lines = test_data.split('\n')
test_soln_lines = test_data_solution.split('\n')


train_descriptions = []
y_train = []

for line in train_lines:
    parts = line.split(" ::: ")
    if len(parts) == 4:
        train_descriptions.append(parts[3])
        y_train.append(parts[2])


test_descriptions = []

for line in test_soln_lines:
    parts = line.split(" ::: ")
    if len(parts) == 3:
        test_descriptions.append(parts[3])





In [7]:
import string
import nltk
from nltk.corpus import stopwords


nltk.download('stopwords')

# Define a function for text preprocessing
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()

    # Remove punctuation
    text = ''.join([char for char in text if char not in string.punctuation])

    # Tokenize the text (split it into words)
    words = text.split()

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Join the words back into a single string
    processed_text = ' '.join(words)

    return processed_text


X_train = [preprocess_text(description) for description in train_descriptions]
X_test = [preprocess_text(description) for description in test_descriptions]




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:


# Split the test data into individual movie descriptions
test_lines = test_data_solution.split('\n')

# Initialize a list to store test genres
y_test = []

# Extract the genres from the test data
for line in test_lines:
    parts = line.split(" ::: ")
    if len(parts) == 4:
        y_test.append(parts[2])



In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=38)


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a TF-IDF vectorizer with your desired settings
tfidf_vectorizer = TfidfVectorizer(max_features=10000,
                                   ngram_range=(1, 2),
                                   min_df=1,
                                   max_df=0.98)

# Fit and transform the vectorizer on your preprocessed training descriptions
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

X_test_tfidf = tfidf_vectorizer.transform(X_test)



In [11]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Create a Multinomial Naive Bayes classifier
naive_bayes_classifier = MultinomialNB(alpha=0.1)

# Train the classifier on the training data
naive_bayes_classifier.fit(X_train_tfidf, y_train)

# Make predictions on the test data
y_test_pred = naive_bayes_classifier.predict(X_test_tfidf)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_test_pred)
report = classification_report(y_test, y_test_pred)



print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)




Accuracy: 0.5565802822097206
Classification Report:
               precision    recall  f1-score   support

      action       0.50      0.21      0.30       252
       adult       0.48      0.13      0.21       104
   adventure       0.38      0.07      0.12       176
   animation       0.50      0.03      0.05       109
   biography       0.00      0.00      0.00        54
      comedy       0.52      0.48      0.50      1491
       crime       0.29      0.02      0.04       103
 documentary       0.62      0.88      0.73      2664
       drama       0.49      0.80      0.61      2683
      family       0.85      0.07      0.13       159
     fantasy       0.33      0.02      0.03        56
   game-show       1.00      0.57      0.72        44
     history       0.00      0.00      0.00        48
      horror       0.67      0.48      0.56       430
       music       0.64      0.46      0.54       149
     musical       0.00      0.00      0.00        70
     mystery       0.00     

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
